# Suomi 24 data preprocessing

In [1]:
import re
import json
import pandas as pd

In [2]:
data = pd.read_csv('https://people.arcada.fi/~parland/suomi24~16-22~subset/posts-16-22.csv.gz', sep=';', compression='gzip')

In [3]:
data.drop(columns="id", inplace=True, axis=1)

In [4]:
labels_to_keep = [7,3,1,2,19,0,9]
data = data[data["class"].isin(labels_to_keep)]

data["class"].value_counts()

7     6737
3     5799
1     4455
2     4333
19    4124
0     3615
9     3166
Name: class, dtype: int64

In [5]:
data.rename(columns={"class":"label"}, inplace=True)

In [6]:
data.head()
#data = pd.concat[data, data[class]]

,topic,article,label
1,Rabbitin pultit pomppailee,Moikka kaikki\n\nKävin juuri hakemassa 1/2016 ...,19
2,Skodapöpö - erikoinen elukka,Vuoden 2015 8 erikoisinta eläinlöytöä. Yksi ...,19
5,Rabbit on paska,Katsastin just 1/2016 Rabbitin. En päästänyt l...,19
6,Luonnonilmiö El Nino,IS kertoo El Ninon sekoittaneen joulukuun lopu...,1
7,Raksutus takapuskurissa,Moi\n\nSain 2016 Skodan ja ihmettelin kun taka...,19


In [7]:
with open("data/abbreviations.json", encoding="utf-8") as f:
    file_content = f.read()

REPLACERS = json.loads(file_content)

In [8]:
# Replace abbreviations
#data["label"].astype(str).str.replace('[...…]','').str.split().apply(lambda x: ' '.join([replacers.get(e, e) for e in x]))
#data["article"].astype(str).str.replace('[...…]','').str.split().apply(lambda x: ' '.join([replacers.get(e, e) for e in x]))


In [9]:
def acronym(s, dct):
    return " ".join([dct.get(i.lower(), i) for i in s.split()])

In [10]:
def clean_text(text):
    text = acronym(text, REPLACERS)
    #text = {rf'(?<!\S){re.escape(k)}(?!\S)': v for k, v in replacers.items()}
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub("[0-9]", " ", text)
    text = re.sub("\W+", " ", text)
    return text


In [11]:
data["topic"] = data["topic"].apply(lambda row: clean_text(row))
data["article"] = data["article"].apply(lambda row: clean_text(row))

In [12]:
data["text"] = data["topic"] + " " + data["article"]
data.drop(columns=["topic", "article"], inplace=True, axis=1)

data.head()

,label,text
1,19,Rabbitin pultit pomppailee Moikka kaikki Kävin...
2,19,Skodapöpö erikoinen elukka Vuoden erikoisinta ...
5,19,Rabbit on paska Katsastin just Rabbitin En pää...
6,1,Luonnonilmiö El Nino IS kertoo El Ninon sekoit...
7,19,Raksutus takapuskurissa Moi Sain Skodan ja ihm...


In [13]:
train_data = data.sample(frac=0.95, random_state=200)
test_data = data.drop(train_data.index)

In [14]:
print(len(test_data))
print(len(train_data))

1611
30618


In [15]:
test_data

,label,text
27,7,Järjestelmävalvojatilin muuttaminen Haluan muu...
28,1,Ilmastojalanjälkeni tulee olemaa edelleen suur...
52,1,Ennätyslumisade tukki Kanadan kulkuväylät vaik...
215,1,Huolestuttava tilanne Mitä ovat viherpellet aj...
218,19,Käyttökieltoon Heh heh skeidajullit vinkuu et...
...,...,...
54131,2,Toyota jäi taas jumboksi Kaikki muut aasialais...
54138,0,Bensiini eur l vastaan sähkö eur kWh Kyllä nyt...
54155,0,Suomesta vedyntuotannon kärkimaa Suomessa vihr...
54168,2,Tässä on osa Toyotan saavutuksista Toyotan saa...


In [16]:
# Rename class to label and classes to indexes
# possible_labels = test_data["label"].unique()
# 
# label_dict = {}
# for idx, possible_label in enumerate(possible_labels):
#     label_dict[possible_label] = idx
# 
# test_data["label"] = test_data["label"].replace(label_dict)

In [17]:
train_data.to_csv("data/preprocessed_data.csv", index=False)

In [18]:
test_data.to_csv("data/test.csv", index=False)

In [19]:
train_data["label"].value_counts()

7     6387
3     5522
1     4231
2     4131
19    3928
0     3420
9     2999
Name: label, dtype: int64

In [20]:
test_data["label"].value_counts()

7     350
3     277
1     224
2     202
19    196
0     195
9     167
Name: label, dtype: int64